### Set Deskew & Attenuation 

===============Instrument in need=================

* MSO64B
* AFG31101

三支探棒接上相同的訊號源去做dewkew、attenuation校正

In [1]:
#引入需要的包裝

import pyvisa
import time

### AFG31101

===============Function List=================

* Set weveform type
* Set frequency
* Set amplitude
* Set output state
* Set amplitude limit
  


In [2]:
class AFG_31101():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to AFG_31101.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("AFG_31101 connection closed.")
        else:
            print("No instrument to close.")

    def set_waveform(self, shape):
        #設定波形類型
        if self.instrument:
            self.instrument.write(f"SOURce1:FUNCtion {shape}")
            time.sleep(0.05)
            print(f"set function shape be {shape}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_frequency(self, freq_amplitude, freq_unit):
        #設定波形頻率
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed {freq_amplitude} {freq_unit}")
            time.sleep(0.05)
            print(f"set frequency be {freq_amplitude}{freq_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_frequency(self):
        #查詢波形頻率
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed?")
            time.sleep(0.05)
            return self.instrument.read()
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_amplitude(self, amp_magnitude, amp_unit):
        #設定波形振福
        if self.instrument:
            self.instrument.write(f"SOURce1:VOLTage:LEVel:IMMediate:AMPLitude {amp_magnitude}{amp_unit}")
            time.sleep(0.05)
            print(f"set amplitude be {amp_magnitude}{amp_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")
    
    def output_on(self):
        #輸出on
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe ON")
            time.sleep(0.05)
            print(f"turn output on")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def output_off(self):
        #輸出off
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe OFF")
            time.sleep(0.05)
            print(f"turn output off")
        else:
            raise Exception("Instrument not connected. Please connect first.")

### MSO64B

===============Function List=================

* Auto set horizontal scale
* Auto set Gating type
* Auto set cursor positon
* Auto set bandwidth limit
* Auto set Acquistion mode & numbers of waveform
* Set measurements[AMP、MEAN、DELAY]
* Set math function

若要串接其他儀器 一樣使用class 定義其他儀器的function

In [3]:
class MSO_64B():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to MSO64B.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("MSO64B connection closed.")
        else:
            print("No instrument to close.")

    def clear(self):
        if self.instrument:
            self.instrument.write(f':CLEAR')
            time.sleep(0.05)  # 在命令之間暫停0.5
            print(f"clear acquistions, measuremetns, adn waveforms")
        else:
            raise Exception("Instrument not connected. Please connect first.")
            
    def measure_delay(self, meas, source1, source2):
        if self.instrument:
            # 將命令連接在一起，減少單獨發送命令的次數
            type_CMD = f":MEASUrement:{meas}:TYPE DELAY" #測量類型CMD
            source1_CMD = f":MEASUrement:{meas}:SOURCE1 {source1}" #訊號源1CMD
            source2_CMD = f":MEASUrement:{meas}:SOURCE2 {source2}" #訊號源2CMD
            command = (f"{type_CMD};{source1_CMD};{source2_CMD};") #完整叫出DELAY的CMD
            self.instrument.write(command) #寫入儀器
            time.sleep(0.05) # 在命令之間暫停0.1秒
            print(f"Add {meas} to measure delay of {source1} to {source2}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def measure_amplitude(self, meas, source):
        if self.instrument:
            type_CMD = f":MEASUrement:{meas}:TYPE AMPLitude" #測量類型CMD
            source_CMD = f":MEASUrement:{meas}:SOURCE {source}" #訊號源CMD
            command = f"{type_CMD};{source_CMD}" #完整叫出AMPLITUDE的CMD
            self.instrument.write(command) #寫入儀器
            time.sleep(0.05) # 在命令之間暫停0.1秒
            print(f"Add {meas} to measure Amplitude of {source}")
        else:
            raise Exception("Instrument not connected. Please connect first.")        
          
    def query_scale(self):
        if self.instrument:
            return self.instrument.query(f":MEASUrement:MEAS10:RESUlts:CURRentacq:MAXimum?")
            time.sleep(0.05) # 在命令之間暫停0.1秒
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def autoset_scale(self, horizon_scale):
        if self.instrument:
            self.instrument.write(f':HORIZONTAL:MODE:SCALE {horizon_scale}')
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"set Horizontal Scale to {horizon_scale}s/div")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def cursor_control(self, gating_type, cursor_A_Position, cursor_B_Position):
        if self.instrument:
            self.instrument.write(f'MEASUrement:GATing {gating_type}')#設定measurement中Gating的項目
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"take measurements on the portion of the waveform between the {gating_type}")
            self.instrument.write(f':DISplay:WAVEView1:CURSor:CURSOR1:STATE ON')
            self.instrument.write(f':DISplay:WAVEView1:CURSor:CURSOR1:WAVEform:APOSition {cursor_A_Position}')
            self.instrument.write(f':DISplay:WAVEView1:CURSor:CURSOR1:WAVEform:BPOSition {cursor_B_Position}')
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"set cursor range be {cursor_A_Position} to {cursor_B_Position}")
        else:
            raise Exception("Instrument not connected. Please connect first.")
            
    def result_table(self):
        if self.instrument:
            self.instrument.write(f':MEASTABle:ADDNew "TABLE1"')
            time.sleep(0.05)  # 在命令之間暫停0.5
            print(f"add measurent result table")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_population(self):
        if self.instrument:
            self.instrument.write(f'MEASUrement:MEAS4:RESUlts:ALLAcqs:POPUlation?')
            time.sleep(0.05)
            return int(self.instrument.read())
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_mean_value(self, meas):
        if self.instrument:                    
            self.instrument.write(f'MEASUrement:{meas}:RESUlts:ALLAcqs:MEAN?')
            time.sleep(0.05)
            return float(self.instrument.read())
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def set_deskew(self, delay_ch1, delay_ch2):
        if self.instrument:
            self.instrument.write(f'CH2:DESKEW {delay_ch1}')
            print(f"deskew ch2: {delay_ch1}")
            time.sleep(0.05)
            self.instrument.write(f'CH3:DESKEW {delay_ch2}')
            print(f"deskew ch3: {delay_ch2}")
            time.sleep(0.05)
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_external(self, ratio, meas):
        if self.instrument:
            self.instrument.write(f'{meas}:PROBEFunc:EXTAtten {ratio}')
            print(f"set external attenuation of {meas} be {ratio}")
            time.sleep(0.05)
        else:
            raise Exception("Instrument not connected. Please connect first.")

In [4]:
def set_horizonscale_cursor(afg, mso):
    """
    設置示波器的光標範圍和水平尺度。
    :param afg: AFG_31101 實例
    :param mso: MSO_64B 實例
    :return: cursor_A位置, cursor_B位置
    """
    query_freq = afg.query_frequency()
    freq = float(query_freq)
    print(freq)
    horizon_scale = (1 / freq) * (4 / 10)
    print(horizon_scale)
    cursor_A_position = -1.5 * (1 / freq)  # 設cursor_A為負2波形週期sec (老師說cursor四個周期為佳)
    cursor_B_position = 1.5 * (1 / freq)  # 設cursor_B為正2波形週期sec

    return horizon_scale, cursor_A_position, cursor_B_position

In [5]:
def wait_for_population(mso, threshold=100):
    while True: #重複查詢採樣次數(population)
        pop_num = mso.query_population()
        print(f"Current population: {pop_num}") #列出每0.5秒的pop次數
        if pop_num >= threshold: #當pop次數大於等於100，結束while迴圈
            print("Poplution採樣數量已超過100")
            break #跳脫迴圈
        time.sleep(0.5)  # 防止過於頻繁的查詢，調整每0.5秒查詢一次

In [6]:
def save_query_values(mso):
    delay_values = {} #新增一空字典放query來的delay值
    amplitude_values = {} #新增一空字典放query來的amplitude值
    
    for meas in ['MEAS1', 'MEAS2']: #當measrement狀態為MEAS1及MEAS2時
        delay = mso.query_mean_value(meas) #接收query_mean_value return回來的值
        delay_values[meas] = delay #把接收到的值丟到delay空字典中
        if meas == 'MEAS1': 
            print(f"The delay of ch1 to ch2 is {delay_values['MEAS1']} sec") #列出CH1 to CH2的delay值
        else:
            print(f"The delay of ch1 to ch3 is {delay_values['MEAS2']} sec") #列出CH1 to CH3的delay值
        
    for meas in ['MEAS3', 'MEAS4', 'MEAS5']: #當measrement狀態為MEAS3、MEAS4及MEA5時
        amplitude = mso.query_mean_value(meas) #接收query_mean_value return回來的值
        amplitude_values[meas] = amplitude
        if meas == 'MEAS3': 
            print(f"The amplitude of ch1 is {amplitude_values['MEAS3']} V") #列出CH1的amplitude值
        elif meas == 'MEAS4':
            print(f"The amplitude of ch2 is {amplitude_values['MEAS4']} V") #列出CH2的amplitude值
        else:
            print(f"The amplitude of ch3 is {amplitude_values['MEAS5']} V") #列出CH3的amplitude值
    
    return delay_values, amplitude_values

In [7]:
def calculate_attenuation(amplitude_values):
    max_meas = max(amplitude_values, key=amplitude_values.get)
    max_value = amplitude_values[max_meas]
    attenuation_ratios = {}
    
    for meas in ['MEAS3', 'MEAS4', 'MEAS5']: #跑過三個測量AMP的測量項
        if meas != max_meas:  #判斷非為最大測量項(max_meas)的另外兩個通道
            ratio = max_value / amplitude_values[meas] #將最大amp值除以目前通道測量的amp算出比例
            attenuation_ratios[meas] = ratio #把算出的比例丟入attenuation_ratio此字典
            print(f"The amplitude ratio of {max_meas} to {meas} is {ratio}") 
    
    return attenuation_ratios, max_meas #將算好後的attenuation_ratio字典及判斷後的最大測量通道回傳

In [8]:
def adjust_deskew_and_attenuation(mso, delay_values, attenuation_ratios, max_meas):
    #調整deskew
    mso.set_deskew(delay_values['MEAS1'], delay_values['MEAS2'])

    #定義一個字典，將最大測量通道(max_meas)對其他通道的關係
    channel_mapping = {
        'MEAS3': {'MEAS4': 'CH2', 'MEAS5': 'CH3'},
        'MEAS4': {'MEAS3': 'CH1', 'MEAS5': 'CH3'},
        'MEAS5': {'MEAS3': 'CH1', 'MEAS4': 'CH2'}
    }
    # attenuqtion_ratio包含[最大AMP]之通道對另外兩個通道之[AMP]比例
    # 调整attenuation
    for meas, ratio in attenuation_ratios.items():
        target_channel = channel_mapping.get(max_meas, {}).get(meas, meas)
        mso.set_external(ratio, target_channel)

In [9]:
def main():
    # =========================Parmeters===============================
    #===================AFG_31101 parameter==============
    """
    shape:{SINusoid|SQUare|PULSe|RAMP|PRNoise|DC|SINC|GAUSsian|LORentz|ERISe|EDECay|HAVersine
     |EMEMory[1]|EMEMory2|EFILe} 
    amplitude:
        magnitude:output amplitude
        <units>::=[VPP | VRMS | DBM]
    frequency:
        magnitude:output frequency
        <units>::=[Hz | kHz | MHz]
    output_state:{ON|OFF}
    """
    #設置訊號參數
    shape = 'SINusoid'
    frequency = {'magnitude': '1','unit': 'MHz'}
    amplitude = {'magnitude': '400E-3','unit': 'VPP'}
    output_state = "ON"
    
    #=================MSO64B parameter====================
    """
    channel:{CH1 | CH2 | CH3 | CH4 }[page:2-541]
    measure type:{AMPLITUDE | FREQUENCY | MAXIMUM | MEAN | MINIMUM | PERIOD | PHASE 
    | PK2PK | RMS } [page:2-717]
    math_num:{MATH1 | MATH2 | MATH3 | MATH4 | ...}[page:2-672]
    math_function:{first letter should be capital then calculate}[page:2-677]
    gating_type:{NONE | SCREEN | CURSor | LOGic | SEARch | TIMe }[page:2-754]
    """

    #新增測量delay的measurement
    measure_delays = [
        {'measure': 'MEAS1','source1': 'CH1','source2': 'CH2'}, 
        {'measure': 'MEAS2','source1': 'CH1','source2': 'CH3'},
    ]

    #新增測量amplitude的measurement
    measure_amps = [
        {'measure': 'MEAS3','source': 'CH1'},
        {'measure': 'MEAS4','source': 'CH2'},
        {'measure': 'MEAS5','source': 'CH3'}        
    ]  

    gating_type = 'CURSor'
    
    #====================Instrument Connect Test===========================
    #=====Address of instruments
    afg_instrument_address = 'USB0::0x0699::0x0359::C016504::INSTR' #AFG的位址
    mso_instrument_address = 'USB0::0x0699::0x0530::C048992::INSTR' #MSO的位址

    #=====connect instruments
    afg = AFG_31101(afg_instrument_address) #連接AFG_31101
    mso = MSO_64B(mso_instrument_address) #連接MSO64B
     #若有其他儀器串接下去...

    #============================Auto Test================================= 
    # TO DO:寫For迴圈自動測量的區域
    
    #================AFG_31101 control===================
    afg.set_waveform(shape) #設置訊號類

    freq_magnitude = frequency['magnitude']
    freq_unit = frequency['unit']
    afg.set_frequency(freq_magnitude,freq_unit) #設置訊號頻率

    amp_magnitude = amplitude['magnitude']
    amp_unit = amplitude['unit']
    afg.set_amplitude(amp_magnitude,amp_unit) #設置訊號振幅

    if output_state == 'ON': #判斷書出狀態，選擇def
        afg.output_on()
    else:
        afg.output_off()

    #=====================MSO64B control================
    horizon_scale, cursor_A_position, cursor_B_position = set_horizonscale_cursor(afg, mso)
    
    #Autoset scale
    mso.autoset_scale(horizon_scale) #先設定好horizon scale  
    
    mso.cursor_control(gating_type, cursor_A_position, cursor_B_position)
            
    for measure_delay in measure_delays: #for迴圈跑字典的多個狀態
        #抓起measure_delays的參數
        meas = measure_delay['measure']
        source1 = measure_delay['source1']
        source2 = measure_delay['source2']
        mso.measure_delay(meas,source1,source2) #將參數丟入measure_delay測量delay值
    
    for measure_amp in measure_amps: #for迴圈跑字典的多個狀態
        #抓起measure_amps的參數
        meas = measure_amp['measure']
        source = measure_amp['source']
        mso.measure_amplitude(meas,source) #將參數丟入measure_amplitude測量delay值

    mso.result_table()#新增measurement result table的介面
    
    mso.clear() #將pop歸零
    time.sleep(0.5)
    #mean_measure = 'MEAS1'  # 以測量1的pop為基準
    wait_for_population(mso) #pop次數超過100繼續往下

    delay_values, amplitude_values = save_query_values(mso) #從save_query_values抓取delay值和amplitude值

    attenuation_ratios, max_meas = calculate_attenuation(amplitude_values)
    print(attenuation_ratios, f"max measurement:{max_meas}")

    adjust_deskew_and_attenuation(mso, delay_values, attenuation_ratios, max_meas)

    mso.clear() #將pop歸零
    time.sleep(0.5)
    wait_for_population(mso) #pop次數超過100繼續往下
    afg.output_off() #deskew & attenuation後將訊號供應AFG輸出關閉
        
    # ============================ Close Instrument=================================
    afg.close_instrument()
    mso.close_instrument()
    
if __name__ == "__main__":
    main()

Connected to AFG_31101.
TEKTRONIX,AFG31101,C016504,SCPI:99.0 FV:1.6.1

Connected to MSO64B.
TEKTRONIX,MSO64B,C048992,CF:91.1CT FV:1.44.3.433

set function shape be SINusoid
set frequency be 1MHz
set amplitude be 400E-3VPP
turn output on
1000000.0
4e-07
set Horizontal Scale to 4e-07s/div
take measurements on the portion of the waveform between the CURSor
set cursor range be -1.5e-06 to 1.5e-06
Add MEAS1 to measure delay of CH1 to CH2
Add MEAS2 to measure delay of CH1 to CH3
Add MEAS3 to measure Amplitude of CH1
Add MEAS4 to measure Amplitude of CH2
Add MEAS5 to measure Amplitude of CH3
add measurent result table
clear acquistions, measuremetns, adn waveforms
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 6
Current population: 18
Current population: 30
Cu